In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Cloning Shang Jing's GitHub repository to extract the Auto Phrase code, which aids in extracting high-quality text for accurately capturing the context's essential words.

In [ ]:
! git clone https://github.com/shangjingbo1226/AutoPhrase.git

Cloning into 'AutoPhrase'...
remote: Enumerating objects: 967, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 967 (delta 66), reused 124 (delta 60), pack-reused 830
Receiving objects: 100% (967/967), 199.80 MiB | 10.98 MiB/s, done.
Resolving deltas: 100% (438/438), done.
Updating files: 100% (204/204), done.


In [ ]:
%cd /content/drive/MyDrive/AutoPhrase/
! bash auto_phrase.sh

/content/drive/MyDrive/AutoPhrase
===Compilation===
===Tokenization===

real	0m41.948s
user	1m3.242s
sys	0m1.482s
Detected Language: EN
Current step: Tokenizing wikipedia phrases...
No provided expert labels.
===AutoPhrasing===
=== Current Settings ===
Iterations = 2
Minimum Support Threshold = 10
Maximum Length Threshold = 6
POS-Tagging Mode Disabled
Discard Ratio = 0.050000
Number of threads = 10
Labeling Method = DPDN
	Auto labels from knowledge bases
	Max Positive Samples = -1
Loading data...
# of total tokens = 13325086
max word token id = 119699
# of documents = 50001
# of distinct POS tags = 0
Mining frequent phrases...
selected MAGIC = 119701
# of frequent phrases = 340609
Extracting features...
Constructing label pools...
	The size of the positive pool = 18045
	The size of the negative pool = 320290
# truth patterns = 180094
Estimating Phrase Quality...
Segmenting...
Rectifying features...
Estimating Phrase Quality...
Segmenting...
Dumping results...
Done.

real	4m57.746s
user

Clone the Yumeng Github for extracting the code for generating the Category based embeddings.


Generating category-based embeddings offers several advantages:

1.Improved classification: It enhances the model's ability to classify text into specific categories, enabling better accuracy in tasks like topic labeling or sentiment analysis.

2.Contextual understanding: Category-based embeddings capture unique context and nuances within each category, providing more precise representations of text for different domains.

3.Enhanced transfer learning: These embeddings can be used in various downstream NLP tasks, helping models transfer knowledge and perform better on specific categories, improving overall model performance.

In [ ]:
%cd /content/drive/MyDrive/
#! git clone https://github.com/yumeng5/CatE.git

/content/drive/MyDrive


Phrasal segmentation is performed on the datasets using high-quality text, and then the background and contextual words are jointly extracted and combined.

In [ ]:
%cd /content/drive/MyDrive/AutoPhrase/
! bash phrasal_segmentation.sh

/content/drive/MyDrive/AutoPhrase
===Compilation===
===Tokenization===

real	0m18.162s
user	0m26.235s
sys	0m1.381s
Detected Language: EN
===Phrasal Segmentation===
=== Current Settings ===
Segmentation Model Path = model_Movies/DBLP/segmentation.model
After the phrasal segmentation, only following phrases will be highlighted with <phrase> and </phrase>
	Q(multi-word phrases) >= 0.700000
	Q(single-word phrases) >= 1.000000
Length penalty model loaded.
	penalty = 199.805
# of loaded patterns = 48912
# of loaded truth patterns = 198139
Phrasal segmentation finished.
   # of total highlighted quality phrases = 1173121
   # of total processed sentences = 1581322
   avg highlights per sentence = 0.741861

real	0m19.100s
user	0m17.101s
sys	0m0.397s
===Generating Output===

real	0m20.819s
user	0m17.144s
sys	0m1.055s


Process the segmented words by converting them to lowercase and join the main word with background words using underscores to ensure accurate context understanding by the embedding generator.

In [ ]:
import argparse
import os
import csv
import numpy as np
from tqdm import tqdm
from nltk import word_tokenize
import re

def phrase_process():
	f = open(os.path.join('AutoPhrase', 'model_News', "DBLP" , 'segmentation.txt'))
	g = open(args.out_file, 'w')
	for line in tqdm(f):
		doc = ''
		temp = re.split(r'<phrase_Q=\d\.\d+>', line)
		for seg in temp:
			temp2 = seg.split('</phrase>')
			if len(temp2) > 1:
				doc += ("_").join(temp2[0].split(" ")) + temp2[1]
			else:
				doc += temp2[0]
		g.write(doc.strip()+'\n')
	print("Phrase segmented corpus written to {}".format(args.out_file))
	return
def preprocess():
	f = open(os.path.join("AutoPhrase","model_Movies/DBLP","text.txt"))
	docs = f.readlines()
	f_out = open(args.out_file, 'w')
	for doc in tqdm(docs):
		f_out.write(' '.join([w.lower() for w in word_tokenize(doc.strip())]) + '\n')
	return


if __name__=="__main__":

	parser = argparse.ArgumentParser(description='main', formatter_class=argparse.ArgumentDefaultsHelpFormatter)
	parser.add_argument('--mode', type=int)
	parser.add_argument('--dataset', default="News")
	parser.add_argument('--in_file', default='segmentation.txt')
	parser.add_argument('--out_file', default='./AutoPhrase/model_News/DBLP/text.txt')
	args, _ = parser.parse_known_args()


phrase_process()
preprocess()

127600it [00:02, 44581.13it/s]


Phrase segmented corpus written to ./AutoPhrase/model_News/DBLP/text.txt


100%|██████████| 50001/50001 [01:53<00:00, 442.09it/s]


In [ ]:
%cd /content/drive/MyDrive/CatE

/content/drive/MyDrive/CatE


In [ ]:
!chmod +x ./src/cate


In [ ]:
! bash run.sh

make: 'cate' is up to date.
Starting training using file ./datasets/News/News_processed.txt
Training with specificity; Specificity values output to file ./datasets/News/emb_category_spec.txt
Reading topics from file ./datasets/News/category.txt
Vocab size: 50110
Words in train file: 12837739
Loading embedding from file word2vec_100.txt
In vocab: 35656
Read 4 topics
politics	
sports	
business	
technology	
Pre-training for 2 epochs, in total 2 + 10 = 12 epochs
Alpha: 0.024969  Progress: 0.13%  Words/thread/sec: 42.63k  ^C


In [2]:
%cd /content/drive/MyDrive/WeSTClass

/content/drive/MyDrive/WeSTClass


In [ ]:
! git clone https://github.com/yumeng5/WeSTClass.git

Cloning into 'WeSTClass'...
remote: Enumerating objects: 75, done.
remote: Total 75 (delta 0), reused 0 (delta 0), pack-reused 75
Receiving objects: 100% (75/75), 61.32 MiB | 11.68 MiB/s, done.
Resolving deltas: 100% (30/30), done.
Updating files: 100% (20/20), done.


In [3]:
! pip3 install -r requirements.txt

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
!pip install spherecluster


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 3.4 MB/s eta 0:00:00


In [5]:
!pip install joblib


In [7]:
import warnings

import numpy as np
import scipy.sparse as sp

from sklearn.cluster import KMeans
from sklearn.cluster._kmeans import (
    _check_sample_weight,
    _labels_inertia,
    _tolerance,
)
from sklearn.utils import check_array, check_random_state
from sklearn.utils.validation import _num_samples
from sklearn.cluster import _kmeans
from sklearn.preprocessing import normalize
from joblib import Parallel, delayed
from sklearn.utils.extmath import row_norms, squared_norm


def _spherical_kmeans_single_lloyd(
    X,
    n_clusters,
    sample_weight=None,
    max_iter=300,
    init="k-means++",
    verbose=False,
    x_squared_norms=None,
    random_state=None,
    tol=1e-4,
    precompute_distances=True,
):
    """
    Modified from sklearn.cluster.k_means_.k_means_single_lloyd.
    """
    random_state = check_random_state(random_state)

    sample_weight = _check_sample_weight(X, sample_weight)

    best_labels, best_inertia, best_centers = None, None, None

    # init
    centers = KMeans._init_centroids(
        X, n_clusters, init, random_state=random_state, x_squared_norms=x_squared_norms
    )
    if verbose:
        print("Initialization complete")

    # Allocate memory to store the distances for each sample to its
    # closer center for reallocation in case of ties
    distances = np.zeros(shape=(X.shape[0],), dtype=X.dtype)

    # iterations
    for i in range(max_iter):
        centers_old = centers.copy()

        # labels assignment
        # TODO: _labels_inertia should be done with cosine distance
        #       since ||a - b|| = 2(1 - cos(a,b)) when a,b are unit normalized
        #       this doesn't really matter.
        labels, inertia = _labels_inertia(
            X,
            sample_weight,
            x_squared_norms,
            centers,
            precompute_distances=precompute_distances,
            distances=distances,
        )

        # computation of the means
        if sp.issparse(X):
            centers = _k_means._centers_sparse(
                X, sample_weight, labels, n_clusters, distances
            )
        else:
            centers = _k_means._centers_dense(
                X, sample_weight, labels, n_clusters, distances
            )

        # l2-normalize centers (this is the main contibution here)
        centers = normalize(centers)

        if verbose:
            print("Iteration %2d, inertia %.3f" % (i, inertia))

        if best_inertia is None or inertia < best_inertia:
            best_labels = labels.copy()
            best_centers = centers.copy()
            best_inertia = inertia

        center_shift_total = squared_norm(centers_old - centers)
        if center_shift_total <= tol:
            if verbose:
                print(
                    "Converged at iteration %d: "
                    "center shift %e within tolerance %e" % (i, center_shift_total, tol)
                )
            break

    if center_shift_total > 0:
        # rerun E-step in case of non-convergence so that predicted labels
        # match cluster centers
        best_labels, best_inertia = _labels_inertia(
            X,
            sample_weight,
            x_squared_norms,
            best_centers,
            precompute_distances=precompute_distances,
            distances=distances,
        )

    return best_labels, best_inertia, best_centers, i + 1


def spherical_k_means(
    X,
    n_clusters,
    sample_weight=None,
    init="k-means++",
    n_init=10,
    max_iter=300,
    verbose=False,
    tol=1e-4,
    random_state=None,
    copy_x=True,
    n_jobs=1,
    algorithm="auto",
    return_n_iter=False,
):
    """Modified from sklearn.cluster.k_means_.k_means.
    """
    if n_init <= 0:
        raise ValueError(
            "Invalid number of initializations."
            " n_init=%d must be bigger than zero." % n_init
        )
    random_state = check_random_state(random_state)

    if max_iter <= 0:
        raise ValueError(
            "Number of iterations should be a positive number,"
            " got %d instead" % max_iter
        )

    best_inertia = np.infty
    # avoid forcing order when copy_x=False
    order = "C" if copy_x else None
    X = check_array(
        X, accept_sparse="csr", dtype=[np.float64, np.float32], order=order, copy=copy_x
    )
    # verify that the number of samples given is larger than k
    if _num_samples(X) < n_clusters:
        raise ValueError(
            "n_samples=%d should be >= n_clusters=%d" % (_num_samples(X), n_clusters)
        )
    tol = _tolerance(X, tol)

    if hasattr(init, "__array__"):
        init = check_array(init, dtype=X.dtype.type, order="C", copy=True)
        KMeans._validate_center_shape(X, n_clusters, init)


        if n_init != 1:
            warnings.warn(
                "Explicit initial center position passed: "
                "performing only one init in k-means instead of n_init=%d" % n_init,
                RuntimeWarning,
                stacklevel=2,
            )
            n_init = 1

    # precompute squared norms of data points
    x_squared_norms = row_norms(X, squared=True)

    if n_jobs == 1:
        # For a single thread, less memory is needed if we just store one set
        # of the best results (as opposed to one set per run per thread).
        for it in range(n_init):
            # run a k-means once
            labels, inertia, centers, n_iter_ = _spherical_kmeans_single_lloyd(
                X,
                n_clusters,
                sample_weight,
                max_iter=max_iter,
                init=init,
                verbose=verbose,
                tol=tol,
                x_squared_norms=x_squared_norms,
                random_state=random_state,
            )

            # determine if these results are the best so far
            if best_inertia is None or inertia < best_inertia:
                best_labels = labels.copy()
                best_centers = centers.copy()
                best_inertia = inertia
                best_n_iter = n_iter_
    else:
        # parallelisation of k-means runs
        seeds = random_state.randint(np.iinfo(np.int32).max, size=n_init)
        results = Parallel(n_jobs=n_jobs, verbose=0)(
            delayed(_spherical_kmeans_single_lloyd)(
                X,
                n_clusters,
                sample_weight,
                max_iter=max_iter,
                init=init,
                verbose=verbose,
                tol=tol,
                x_squared_norms=x_squared_norms,
                # Change seed to ensure variety
                random_state=seed,
            )
            for seed in seeds
        )

        # Get results with the lowest inertia
        labels, inertia, centers, n_iters = zip(*results)
        best = np.argmin(inertia)
        best_labels = labels[best]
        best_inertia = inertia[best]
        best_centers = centers[best]
        best_n_iter = n_iters[best]

    if return_n_iter:
        return best_centers, best_labels, best_inertia, best_n_iter
    else:
        return best_centers, best_labels, best_inertia


class SphericalKMeans(KMeans):
    """Spherical K-Means clustering

    Modfication of sklearn.cluster.KMeans where cluster centers are normalized
    (projected onto the sphere) in each iteration.

    Parameters
    ----------

    n_clusters : int, optional, default: 8
        The number of clusters to form as well as the number of
        centroids to generate.

    max_iter : int, default: 300
        Maximum number of iterations of the k-means algorithm for a
        single run.

    n_init : int, default: 10
        Number of time the k-means algorithm will be run with different
        centroid seeds. The final results will be the best output of
        n_init consecutive runs in terms of inertia.

    init : {'k-means++', 'random' or an ndarray}
        Method for initialization, defaults to 'k-means++':
        'k-means++' : selects initial cluster centers for k-mean
        clustering in a smart way to speed up convergence. See section
        Notes in k_init for more details.
        'random': choose k observations (rows) at random from data for
        the initial centroids.
        If an ndarray is passed, it should be of shape (n_clusters, n_features)
        and gives the initial centers.

    tol : float, default: 1e-4
        Relative tolerance with regards to inertia to declare convergence

    n_jobs : int
        The number of jobs to use for the computation. This works by computing
        each of the n_init runs in parallel.
        If -1 all CPUs are used. If 1 is given, no parallel computing code is
        used at all, which is useful for debugging. For n_jobs below -1,
        (n_cpus + 1 + n_jobs) are used. Thus for n_jobs = -2, all CPUs but one
        are used.

    random_state : integer or numpy.RandomState, optional
        The generator used to initialize the centers. If an integer is
        given, it fixes the seed. Defaults to the global numpy random
        number generator.

    verbose : int, default 0
        Verbosity mode.

    copy_x : boolean, default True
        When pre-computing distances it is more numerically accurate to center
        the data first.  If copy_x is True, then the original data is not
        modified.  If False, the original data is modified, and put back before
        the function returns, but small numerical differences may be introduced
        by subtracting and then adding the data mean.

    normalize : boolean, default True
        Normalize the input to have unnit norm.

    Attributes
    ----------

    cluster_centers_ : array, [n_clusters, n_features]
        Coordinates of cluster centers

    labels_ :
        Labels of each point

    inertia_ : float
        Sum of distances of samples to their closest cluster center.
    """

    def __init__(
        self,
        n_clusters=8,
        init="k-means++",
        n_init=10,
        max_iter=300,
        tol=1e-4,
        n_jobs=1,
        verbose=0,
        random_state=None,
        copy_x=True,
        normalize=True,
    ):
        self.n_clusters = n_clusters
        self.init = init
        self.max_iter = max_iter
        self.tol = tol
        self.n_init = n_init
        self.verbose = verbose
        self.random_state = random_state
        self.copy_x = copy_x
        self.n_jobs = n_jobs
        self.normalize = normalize

    def fit(self, X, y=None, sample_weight=None):
        """Compute k-means clustering.

        Parameters
        ----------

        X : array-like or sparse matrix, shape=(n_samples, n_features)

        y : Ignored
            not used, present here for API consistency by convention.

        sample_weight : array-like, shape (n_samples,), optional
            The weights for each observation in X. If None, all observations
            are assigned equal weight (default: None)
        """
        if self.normalize:
            X = normalize(X)

        random_state = check_random_state(self.random_state)

        # TODO: add check that all data is unit-normalized

        self.cluster_centers_, self.labels_, self.inertia_, self.n_iter_ = spherical_k_means(
            X,
            n_clusters=self.n_clusters,
            sample_weight=sample_weight,
            init=self.init,
            n_init=self.n_init,
            max_iter=self.max_iter,
            verbose=self.verbose,
            tol=self.tol,
            random_state=random_state,
            copy_x=self.copy_x,
            n_jobs=self.n_jobs,
            return_n_iter=True,
        )

        return self


In [10]:
import warnings

import numpy as np
import scipy.sparse as sp
from scipy.special import iv  # modified Bessel function of first kind, I_v
from numpy import i0  # modified Bessel function of first kind order 0, I_0
from scipy.special import logsumexp

from sklearn.base import BaseEstimator, ClusterMixin, TransformerMixin
from sklearn.cluster._kmeans import (
    _check_sample_weight,
    _labels_inertia,
    _tolerance,
)
from sklearn.utils.validation import FLOAT_DTYPES
from sklearn.utils.validation import check_is_fitted
from sklearn.utils import check_array, check_random_state, as_float_array
from sklearn.preprocessing import normalize
from sklearn.utils.extmath import squared_norm
from sklearn.metrics.pairwise import cosine_distances
from joblib import Parallel, delayed

from sklearn.cluster import _kmeans



MAX_CONTENTRATION = 1e10


def _inertia_from_labels(X, centers, labels):
    """Compute inertia with cosine distance using known labels.
    """
    n_examples, n_features = X.shape
    inertia = np.zeros((n_examples,))
    for ee in range(n_examples):
        inertia[ee] = 1 - X[ee, :].dot(centers[int(labels[ee]), :].T)

    return np.sum(inertia)


def _labels_inertia(X, centers):
    """Compute labels and inertia with cosine distance.
    """
    n_examples, n_features = X.shape
    n_clusters, n_features = centers.shape

    labels = np.zeros((n_examples,))
    inertia = np.zeros((n_examples,))

    for ee in range(n_examples):
        dists = np.zeros((n_clusters,))
        for cc in range(n_clusters):
            dists[cc] = 1 - X[ee, :].dot(centers[cc, :].T)

        labels[ee] = np.argmin(dists)
        inertia[ee] = dists[int(labels[ee])]

    return labels, np.sum(inertia)


def _vmf_log(X, kappa, mu):
    """Computs log(vMF(X, kappa, mu)) using built-in numpy/scipy Bessel
    approximations.

    Works well on small kappa and mu.
    """
    n_examples, n_features = X.shape
    return np.log(_vmf_normalize(kappa, n_features) * np.exp(kappa * X.dot(mu).T))


def _vmf_normalize(kappa, dim):
    """Compute normalization constant using built-in numpy/scipy Bessel
    approximations.

    Works well on small kappa and mu.
    """
    num = np.power(kappa, dim / 2. - 1.)

    if dim / 2. - 1. < 1e-15:
        denom = np.power(2. * np.pi, dim / 2.) * i0(kappa)
    else:
        denom = np.power(2. * np.pi, dim / 2.) * iv(dim / 2. - 1., kappa)

    if np.isinf(num):
        raise ValueError("VMF scaling numerator was inf.")

    if np.isinf(denom):
        raise ValueError("VMF scaling denominator was inf.")

    if np.abs(denom) < 1e-15:
        raise ValueError("VMF scaling denominator was 0.")

    return num / denom


def _log_H_asymptotic(nu, kappa):
    """Compute the Amos-type upper bound asymptotic approximation on H where
    log(H_\nu)(\kappa) = \int_0^\kappa R_\nu(t) dt.

    See "lH_asymptotic <-" in movMF.R and utility function implementation notes
    from https://cran.r-project.org/web/packages/movMF/index.html
    """
    beta = np.sqrt((nu + 0.5) ** 2)
    kappa_l = np.min([kappa, np.sqrt((3. * nu + 11. / 2.) * (nu + 3. / 2.))])
    return _S(kappa, nu + 0.5, beta) + (
        _S(kappa_l, nu, nu + 2.) - _S(kappa_l, nu + 0.5, beta)
    )


def _S(kappa, alpha, beta):
    """Compute the antiderivative of the Amos-type bound G on the modified
    Bessel function ratio.

    Note:  Handles scalar kappa, alpha, and beta only.

    See "S <-" in movMF.R and utility function implementation notes from
    https://cran.r-project.org/web/packages/movMF/index.html
    """
    kappa = 1. * np.abs(kappa)
    alpha = 1. * alpha
    beta = 1. * np.abs(beta)
    a_plus_b = alpha + beta
    u = np.sqrt(kappa ** 2 + beta ** 2)
    if alpha == 0:
        alpha_scale = 0
    else:
        alpha_scale = alpha * np.log((alpha + u) / a_plus_b)

    return u - beta - alpha_scale


def _vmf_log_asymptotic(X, kappa, mu):
    """Compute log(f(x|theta)) via Amos approximation

        log(f(x|theta)) = theta' x - log(H_{d/2-1})(\|theta\|)

    where theta = kappa * X, \|theta\| = kappa.

    Computing _vmf_log helps with numerical stability / loss of precision for
    for large values of kappa and n_features.

    See utility function implementation notes in movMF.R from
    https://cran.r-project.org/web/packages/movMF/index.html
    """
    n_examples, n_features = X.shape
    log_vfm = kappa * X.dot(mu).T + -_log_H_asymptotic(n_features / 2. - 1., kappa)

    return log_vfm


def _log_likelihood(X, centers, weights, concentrations):
    if len(np.shape(X)) != 2:
        X = X.reshape((1, len(X)))

    n_examples, n_features = np.shape(X)
    n_clusters, _ = centers.shape

    if n_features <= 50:  # works up to about 50 before numrically unstable
        vmf_f = _vmf_log
    else:
        vmf_f = _vmf_log_asymptotic

    f_log = np.zeros((n_clusters, n_examples))
    for cc in range(n_clusters):
        f_log[cc, :] = vmf_f(X, concentrations[cc], centers[cc, :])

    posterior = np.zeros((n_clusters, n_examples))
    weights_log = np.log(weights)
    posterior = np.tile(weights_log.T, (n_examples, 1)).T + f_log
    for ee in range(n_examples):
        posterior[:, ee] = np.exp(posterior[:, ee] - logsumexp(posterior[:, ee]))

    return posterior


def _init_unit_centers(X, n_clusters, random_state, init):
    """Initializes unit norm centers.

    Parameters
    ----------
    X : array-like or sparse matrix, shape=(n_samples, n_features)

    n_clusters : int, optional, default: 8
        The number of clusters to form as well as the number of
        centroids to generate.

    random_state : integer or numpy.RandomState, optional
        The generator used to initialize the centers. If an integer is
        given, it fixes the seed. Defaults to the global numpy random
        number generator.

    init:  (string) one of
        k-means++ : uses sklearn k-means++ initialization algorithm
        spherical-k-means : use centroids from one pass of spherical k-means
        random : random unit norm vectors
        random-orthonormal : random orthonormal vectors
        If an ndarray is passed, it should be of shape (n_clusters, n_features)
        and gives the initial centers.
    """
    n_examples, n_features = np.shape(X)
    if isinstance(init, np.ndarray):
        n_init_clusters, n_init_features = init.shape
        assert n_init_clusters == n_clusters
        assert n_init_features == n_features

        # ensure unit normed centers
        centers = init
        for cc in range(n_clusters):
            centers[cc, :] = centers[cc, :] / np.linalg.norm(centers[cc, :])

        return centers

    elif init == "spherical-k-means":
        labels, inertia, centers, iters = spherical_kmeans._spherical_kmeans_single_lloyd(
            X, n_clusters, x_squared_norms=np.ones((n_examples,)), init="k-means++"
        )

        return centers

    elif init == "random":
        centers = np.random.randn(n_clusters, n_features)
        for cc in range(n_clusters):
            centers[cc, :] = centers[cc, :] / np.linalg.norm(centers[cc, :])

        return centers

    elif init == "k-means++":
        centers = _init_centroids(
            X,
            n_clusters,
            "k-means++",
            random_state=random_state,
            x_squared_norms=np.ones((n_examples,)),
        )

        for cc in range(n_clusters):
            centers[cc, :] = centers[cc, :] / np.linalg.norm(centers[cc, :])

        return centers

    elif init == "random-orthonormal":
        centers = np.random.randn(n_clusters, n_features)
        q, r = np.linalg.qr(centers.T, mode="reduced")

        return q.T

    elif init == "random-class":
        centers = np.zeros((n_clusters, n_features))
        for cc in range(n_clusters):
            while np.linalg.norm(centers[cc, :]) == 0:
                labels = np.random.randint(0, n_clusters, n_examples)
                centers[cc, :] = X[labels == cc, :].sum(axis=0)

        for cc in range(n_clusters):
            centers[cc, :] = centers[cc, :] / np.linalg.norm(centers[cc, :])

        return centers


def _expectation(X, centers, weights, concentrations, posterior_type="soft"):
    """Compute the log-likelihood of each datapoint being in each cluster.

    Parameters
    ----------
    centers (mu) : array, [n_centers x n_features]
    weights (alpha) : array, [n_centers, ] (alpha)
    concentrations (kappa) : array, [n_centers, ]

    Returns
    ----------
    posterior : array, [n_centers, n_examples]
    """
    n_examples, n_features = np.shape(X)
    n_clusters, _ = centers.shape

    if n_features <= 50:  # works up to about 50 before numrically unstable
        vmf_f = _vmf_log
    else:
        vmf_f = _vmf_log_asymptotic

    f_log = np.zeros((n_clusters, n_examples))
    for cc in range(n_clusters):
        f_log[cc, :] = vmf_f(X, concentrations[cc], centers[cc, :])

    posterior = np.zeros((n_clusters, n_examples))
    if posterior_type == "soft":
        weights_log = np.log(weights)
        posterior = np.tile(weights_log.T, (n_examples, 1)).T + f_log
        for ee in range(n_examples):
            posterior[:, ee] = np.exp(posterior[:, ee] - logsumexp(posterior[:, ee]))

    elif posterior_type == "hard":
        weights_log = np.log(weights)
        weighted_f_log = np.tile(weights_log.T, (n_examples, 1)).T + f_log
        for ee in range(n_examples):
            posterior[np.argmax(weighted_f_log[:, ee]), ee] = 1.0

    return posterior


def _maximization(X, posterior, force_weights=None):
    """Estimate new centers, weights, and concentrations from

    Parameters
    ----------
    posterior : array, [n_centers, n_examples]
        The posterior matrix from the expectation step.

    force_weights : None or array, [n_centers, ]
        If None is passed, will estimate weights.
        If an array is passed, will use instead of estimating.

    Returns
    ----------
    centers (mu) : array, [n_centers x n_features]
    weights (alpha) : array, [n_centers, ] (alpha)
    concentrations (kappa) : array, [n_centers, ]
    """
    n_examples, n_features = X.shape
    n_clusters, n_examples = posterior.shape
    concentrations = np.zeros((n_clusters,))
    centers = np.zeros((n_clusters, n_features))
    if force_weights is None:
        weights = np.zeros((n_clusters,))

    for cc in range(n_clusters):
        # update weights (alpha)
        if force_weights is None:
            weights[cc] = np.mean(posterior[cc, :])
        else:
            weights = force_weights

        # update centers (mu)
        X_scaled = X.copy()
        if sp.issparse(X):
            X_scaled.data *= posterior[cc, :].repeat(np.diff(X_scaled.indptr))
        else:
            for ee in range(n_examples):
                X_scaled[ee, :] *= posterior[cc, ee]

        centers[cc, :] = X_scaled.sum(axis=0)

        # normalize centers
        center_norm = np.linalg.norm(centers[cc, :])
        if center_norm > 1e-8:
            centers[cc, :] = centers[cc, :] / center_norm

        # update concentration (kappa) [TODO: add other kappa approximations]
        rbar = center_norm / (n_examples * weights[cc])
        concentrations[cc] = rbar * n_features - np.power(rbar, 3.)
        if np.abs(rbar - 1.0) < 1e-10:
            concentrations[cc] = MAX_CONTENTRATION
        else:
            concentrations[cc] /= 1. - np.power(rbar, 2.)

        # let python know we can free this (good for large dense X)
        del X_scaled

    return centers, weights, concentrations


def _movMF(
    X,
    n_clusters,
    posterior_type="soft",
    force_weights=None,
    max_iter=300,
    verbose=False,
    init="random-class",
    random_state=None,
    tol=1e-6,
):
    """Mixture of von Mises Fisher clustering.

    Implements the algorithms (i) and (ii) from

      "Clustering on the Unit Hypersphere using von Mises-Fisher Distributions"
      by Banerjee, Dhillon, Ghosh, and Sra.

    TODO: Currently only supports Banerjee et al 2005 approximation of kappa,
          however, there are numerous other approximations see _update_params.

    Attribution
    ----------
    Approximation of log-vmf distribution function from movMF R-package.

    movMF: An R Package for Fitting Mixtures of von Mises-Fisher Distributions
    by Kurt Hornik, Bettina Grun, 2014

    Find more at:
      https://cran.r-project.org/web/packages/movMF/vignettes/movMF.pdf
      https://cran.r-project.org/web/packages/movMF/index.html

    Parameters
    ----------
    n_clusters : int, optional, default: 8
        The number of clusters to form as well as the number of
        centroids to generate.

    posterior_type: 'soft' or 'hard'
        Type of posterior computed in exepectation step.
        See note about attribute: self.posterior_

    force_weights : None or array [n_clusters, ]
        If None, the algorithm will estimate the weights.
        If an array of weights, algorithm will estimate concentrations and
        centers with given weights.

    max_iter : int, default: 300
        Maximum number of iterations of the k-means algorithm for a
        single run.

    n_init : int, default: 10
        Number of time the k-means algorithm will be run with different
        centroid seeds. The final results will be the best output of
        n_init consecutive runs in terms of inertia.

    init:  (string) one of
        random-class [default]: random class assignment & centroid computation
        k-means++ : uses sklearn k-means++ initialization algorithm
        spherical-k-means : use centroids from one pass of spherical k-means
        random : random unit norm vectors
        random-orthonormal : random orthonormal vectors
        If an ndarray is passed, it should be of shape (n_clusters, n_features)
        and gives the initial centers.

    tol : float, default: 1e-6
        Relative tolerance with regards to inertia to declare convergence

    n_jobs : int
        The number of jobs to use for the computation. This works by computing
        each of the n_init runs in parallel.
        If -1 all CPUs are used. If 1 is given, no parallel computing code is
        used at all, which is useful for debugging. For n_jobs below -1,
        (n_cpus + 1 + n_jobs) are used. Thus for n_jobs = -2, all CPUs but one
        are used.

    random_state : integer or numpy.RandomState, optional
        The generator used to initialize the centers. If an integer is
        given, it fixes the seed. Defaults to the global numpy random
        number generator.

    verbose : int, default 0
        Verbosity mode.

    copy_x : boolean, default True
        When pre-computing distances it is more numerically accurate to center
        the data first.  If copy_x is True, then the original data is not
        modified.  If False, the original data is modified, and put back before
        the function returns, but small numerical differences may be introduced
        by subtracting and then adding the data mean.
    """
    random_state = check_random_state(random_state)
    n_examples, n_features = np.shape(X)

    # init centers (mus)
    centers = _init_unit_centers(X, n_clusters, random_state, init)

    # init weights (alphas)
    if force_weights is None:
        weights = np.ones((n_clusters,))
        weights = weights / np.sum(weights)
    else:
        weights = force_weights

    # init concentrations (kappas)
    concentrations = np.ones((n_clusters,))

    if verbose:
        print("Initialization complete")

    for iter in range(max_iter):
        centers_prev = centers.copy()

        # expectation step
        posterior = _expectation(
            X, centers, weights, concentrations, posterior_type=posterior_type
        )

        # maximization step
        centers, weights, concentrations = _maximization(
            X, posterior, force_weights=force_weights
        )

        # check convergence
        tolcheck = squared_norm(centers_prev - centers)
        if tolcheck <= tol:
            if verbose:
                print(
                    "Converged at iteration %d: "
                    "center shift %e within tolerance %e" % (iter, tolcheck, tol)
                )
            break

    # labels come for free via posterior
    labels = np.zeros((n_examples,))
    for ee in range(n_examples):
        labels[ee] = np.argmax(posterior[:, ee])

    inertia = _inertia_from_labels(X, centers, labels)

    return centers, weights, concentrations, posterior, labels, inertia


def movMF(
    X,
    n_clusters,
    posterior_type="soft",
    force_weights=None,
    n_init=10,
    n_jobs=1,
    max_iter=300,
    verbose=False,
    init="random-class",
    random_state=None,
    tol=1e-6,
    copy_x=True,
):
    """Wrapper for parallelization of _movMF and running n_init times.
    """
    if n_init <= 0:
        raise ValueError(
            "Invalid number of initializations."
            " n_init=%d must be bigger than zero." % n_init
        )
    random_state = check_random_state(random_state)

    if max_iter <= 0:
        raise ValueError(
            "Number of iterations should be a positive number,"
            " got %d instead" % max_iter
        )

    best_inertia = np.infty
    X = as_float_array(X, copy=copy_x)
    tol = _tolerance(X, tol)

    if hasattr(init, "__array__"):
        init = check_array(init, dtype=X.dtype.type, copy=True)
        _validate_center_shape(X, n_clusters, init)

        if n_init != 1:
            warnings.warn(
                "Explicit initial center position passed: "
                "performing only one init in k-means instead of n_init=%d" % n_init,
                RuntimeWarning,
                stacklevel=2,
            )
            n_init = 1

    # defaults
    best_centers = None
    best_labels = None
    best_weights = None
    best_concentrations = None
    best_posterior = None
    best_inertia = None

    if n_jobs == 1:
        # For a single thread, less memory is needed if we just store one set
        # of the best results (as opposed to one set per run per thread).
        for it in range(n_init):
            # cluster on the sphere
            (centers, weights, concentrations, posterior, labels, inertia) = _movMF(
                X,
                n_clusters,
                posterior_type=posterior_type,
                force_weights=force_weights,
                max_iter=max_iter,
                verbose=verbose,
                init=init,
                random_state=random_state,
                tol=tol,
            )

            # determine if these results are the best so far
            if best_inertia is None or inertia < best_inertia:
                best_centers = centers.copy()
                best_labels = labels.copy()
                best_weights = weights.copy()
                best_concentrations = concentrations.copy()
                best_posterior = posterior.copy()
                best_inertia = inertia
    else:
        # parallelisation of movMF runs
        seeds = random_state.randint(np.iinfo(np.int32).max, size=n_init)
        results = Parallel(n_jobs=n_jobs, verbose=0)(
            delayed(_movMF)(
                X,
                n_clusters,
                posterior_type=posterior_type,
                force_weights=force_weights,
                max_iter=max_iter,
                verbose=verbose,
                init=init,
                random_state=random_state,
                tol=tol,
            )
            for seed in seeds
        )

        # Get results with the lowest inertia
        centers, weights, concentrations, posteriors, labels, inertia = zip(*results)
        best = np.argmin(inertia)
        best_labels = labels[best]
        best_inertia = inertia[best]
        best_centers = centers[best]
        best_concentrations = concentrations[best]
        best_posterior = posteriors[best]
        best_weights = weights[best]

    return (
        best_centers,
        best_labels,
        best_inertia,
        best_weights,
        best_concentrations,
        best_posterior,
    )


class VonMisesFisherMixture(BaseEstimator, ClusterMixin, TransformerMixin):
    """Estimator for Mixture of von Mises Fisher clustering on the unit sphere.

    Implements the algorithms (i) and (ii) from

      "Clustering on the Unit Hypersphere using von Mises-Fisher Distributions"
      by Banerjee, Dhillon, Ghosh, and Sra.

    TODO: Currently only supports Banerjee et al 2005 approximation of kappa,
          however, there are numerous other approximations see _update_params.

    Attribution
    ----------
    Approximation of log-vmf distribution function from movMF R-package.

    movMF: An R Package for Fitting Mixtures of von Mises-Fisher Distributions
    by Kurt Hornik, Bettina Grun, 2014

    Find more at:
      https://cran.r-project.org/web/packages/movMF/vignettes/movMF.pdf
      https://cran.r-project.org/web/packages/movMF/index.html

    Basic sklearn scaffolding from sklearn.cluster.KMeans.

    Parameters
    ----------
    n_clusters : int, optional, default: 8
        The number of clusters to form as well as the number of
        centroids to generate.

    posterior_type: 'soft' or 'hard'
        Type of posterior computed in exepectation step.
        See note about attribute: self.posterior_

    force_weights : None or array [n_clusters, ]
        If None, the algorithm will estimate the weights.
        If an array of weights, algorithm will estimate concentrations and
        centers with given weights.

    max_iter : int, default: 300
        Maximum number of iterations of the k-means algorithm for a
        single run.

    n_init : int, default: 10
        Number of time the k-means algorithm will be run with different
        centroid seeds. The final results will be the best output of
        n_init consecutive runs in terms of inertia.

    init:  (string) one of
        random-class [default]: random class assignment & centroid computation
        k-means++ : uses sklearn k-means++ initialization algorithm
        spherical-k-means : use centroids from one pass of spherical k-means
        random : random unit norm vectors
        random-orthonormal : random orthonormal vectors
        If an ndarray is passed, it should be of shape (n_clusters, n_features)
        and gives the initial centers.

    tol : float, default: 1e-6
        Relative tolerance with regards to inertia to declare convergence

    n_jobs : int
        The number of jobs to use for the computation. This works by computing
        each of the n_init runs in parallel.
        If -1 all CPUs are used. If 1 is given, no parallel computing code is
        used at all, which is useful for debugging. For n_jobs below -1,
        (n_cpus + 1 + n_jobs) are used. Thus for n_jobs = -2, all CPUs but one
        are used.

    random_state : integer or numpy.RandomState, optional
        The generator used to initialize the centers. If an integer is
        given, it fixes the seed. Defaults to the global numpy random
        number generator.

    verbose : int, default 0
        Verbosity mode.

    copy_x : boolean, default True
        When pre-computing distances it is more numerically accurate to center
        the data first.  If copy_x is True, then the original data is not
        modified.  If False, the original data is modified, and put back before
        the function returns, but small numerical differences may be introduced
        by subtracting and then adding the data mean.

    normalize : boolean, default True
        Normalize the input to have unnit norm.

    Attributes
    ----------

    cluster_centers_ : array, [n_clusters, n_features]
        Coordinates of cluster centers

    labels_ :
        Labels of each point

    inertia_ : float
        Sum of distances of samples to their closest cluster center.

    weights_ : array, [n_clusters,]
        Weights of each cluster in vMF distribution (alpha).

    concentrations_ : array [n_clusters,]
        Concentration parameter for each cluster (kappa).
        Larger values correspond to more concentrated clusters.

    posterior_ : array, [n_clusters, n_examples]
        Each column corresponds to the posterio distribution for and example.

        If posterior_type='hard' is used, there will only be one non-zero per
        column, its index corresponding to the example's cluster label.

        If posterior_type='soft' is used, this matrix will be dense and the
        column values correspond to soft clustering weights.
    """

    def __init__(
        self,
        n_clusters=5,
        posterior_type="soft",
        force_weights=None,
        n_init=10,
        n_jobs=1,
        max_iter=300,
        verbose=False,
        init="random-class",
        random_state=None,
        tol=1e-6,
        copy_x=True,
        normalize=True,
    ):
        self.n_clusters = n_clusters
        self.posterior_type = posterior_type
        self.force_weights = force_weights
        self.n_init = n_init
        self.n_jobs = n_jobs
        self.max_iter = max_iter
        self.verbose = verbose
        self.init = init
        self.random_state = random_state
        self.tol = tol
        self.copy_x = copy_x
        self.normalize = normalize

    def _check_force_weights(self):
        if self.force_weights is None:
            return

        if len(self.force_weights) != self.n_clusters:
            raise ValueError(
                (
                    "len(force_weights)={} but must equal "
                    "n_clusters={}".format(len(self.force_weights), self.n_clusters)
                )
            )

    def _check_fit_data(self, X):
        """Verify that the number of samples given is larger than k"""
        X = check_array(X, accept_sparse="csr", dtype=[np.float64, np.float32])
        n_samples, n_features = X.shape
        if X.shape[0] < self.n_clusters:
            raise ValueError(
                "n_samples=%d should be >= n_clusters=%d"
                % (X.shape[0], self.n_clusters)
            )

        for ee in range(n_samples):
            if sp.issparse(X):
                n = sp.linalg.norm(X[ee, :])
            else:
                n = np.linalg.norm(X[ee, :])

            if np.abs(n - 1.) > 1e-4:
                raise ValueError("Data l2-norm must be 1, found {}".format(n))

        return X

    def _check_test_data(self, X):
        X = check_array(X, accept_sparse="csr", dtype=FLOAT_DTYPES, warn_on_dtype=True)
        n_samples, n_features = X.shape
        expected_n_features = self.cluster_centers_.shape[1]
        if not n_features == expected_n_features:
            raise ValueError(
                "Incorrect number of features. "
                "Got %d features, expected %d" % (n_features, expected_n_features)
            )

        for ee in range(n_samples):
            if sp.issparse(X):
                n = sp.linalg.norm(X[ee, :])
            else:
                n = np.linalg.norm(X[ee, :])

            if np.abs(n - 1.) > 1e-4:
                raise ValueError("Data l2-norm must be 1, found {}".format(n))

        return X

    def fit(self, X, y=None):
        """Compute mixture of von Mises Fisher clustering.

        Parameters
        ----------
        X : array-like or sparse matrix, shape=(n_samples, n_features)
        """
        if self.normalize:
            X = normalize(X)

        self._check_force_weights()
        random_state = check_random_state(self.random_state)
        X = self._check_fit_data(X)

        (
            self.cluster_centers_,
            self.labels_,
            self.inertia_,
            self.weights_,
            self.concentrations_,
            self.posterior_,
        ) = movMF(
            X,
            self.n_clusters,
            posterior_type=self.posterior_type,
            force_weights=self.force_weights,
            n_init=self.n_init,
            n_jobs=self.n_jobs,
            max_iter=self.max_iter,
            verbose=self.verbose,
            init=self.init,
            random_state=random_state,
            tol=self.tol,
            copy_x=self.copy_x,
        )

        return self

    def fit_predict(self, X, y=None):
        """Compute cluster centers and predict cluster index for each sample.
        Convenience method; equivalent to calling fit(X) followed by
        predict(X).
        """
        return self.fit(X).labels_

    def fit_transform(self, X, y=None):
        """Compute clustering and transform X to cluster-distance space.
        Equivalent to fit(X).transform(X), but more efficiently implemented.
        """
        # Currently, this just skips a copy of the data if it is not in
        # np.array or CSR format already.
        # XXX This skips _check_test_data, which may change the dtype;
        # we should refactor the input validation.
        return self.fit(X)._transform(X)

    def transform(self, X, y=None):
        """Transform X to a cluster-distance space.
        In the new space, each dimension is the cosine distance to the cluster
        centers.  Note that even if X is sparse, the array returned by
        `transform` will typically be dense.

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            New data to transform.

        Returns
        -------
        X_new : array, shape [n_samples, k]
            X transformed in the new space.
        """
        if self.normalize:
            X = normalize(X)

        check_is_fitted(self, "cluster_centers_")
        X = self._check_test_data(X)
        return self._transform(X)

    def _transform(self, X):
        """guts of transform method; no input validation"""
        return cosine_distances(X, self.cluster_centers_)

    def predict(self, X):
        """Predict the closest cluster each sample in X belongs to.
        In the vector quantization literature, `cluster_centers_` is called
        the code book and each value returned by `predict` is the index of
        the closest code in the code book.

        Note:  Does not check that each point is on the sphere.

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            New data to predict.

        Returns
        -------
        labels : array, shape [n_samples,]
            Index of the cluster each sample belongs to.
        """
        if self.normalize:
            X = normalize(X)

        check_is_fitted(self, "cluster_centers_")

        X = self._check_test_data(X)
        return _labels_inertia(X, self.cluster_centers_)[0]

    def score(self, X, y=None):
        """Inertia score (sum of all distances to closest cluster).

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            New data.

        Returns
        -------
        score : float
            Larger score is better.
        """
        if self.normalize:
            X = normalize(X)

        check_is_fitted(self, "cluster_centers_")
        X = self._check_test_data(X)
        return -_labels_inertia(X, self.cluster_centers_)[1]

    def log_likelihood(self, X):
        check_is_fitted(self, "cluster_centers_")

        return _log_likelihood(
            X, self.cluster_centers_, self.weights_, self.concentrations_
        )


In [11]:
import csv
import numpy as np
import os
import re
import itertools
from collections import Counter
from os.path import join
from nltk import tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
# from transformers import pipeline
# classifier = pipeline("zero-shot-classification", model = "facebook/bart-large-mnli")
def read_file(data_dir, with_evaluation):
    data = []
    target = []
    labels_used = ["bad","good"]
    with open(join(data_dir, 'combined_file_latest.csv'), 'rt', encoding='utf-8') as csvfile:
        csv.field_size_limit(500 * 1024 * 1024)
        reader = csv.reader(csvfile)
        for row in reader:
            if data_dir == './News' :
                doc = row[1] + '. ' + row[2]
                data.append(doc)
                #removed -1
                target.append(int(row[1]))
            elif data_dir == './Movies':
                data.append(row[1])
                #removede -1
                target.append(int(row[0]))
                #target.append(1 if classifier(row[1],labels_used)["labels"][0] == "good" else 0)
    if with_evaluation:
        y = np.asarray(target)
        #print(y)
        y[y==-1]=0
        assert len(data) == len(y)
        assert set(range(len(np.unique(y)))) == set(np.unique(y))
    else:
        y = None
    return data, y


def clean_str(string):
    string = re.sub(r"[^A-Za-z0-9(),.!?_\"\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\"", " \" ", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'m", " \'m", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"\.", " . ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\$", " $ ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()


def preprocess_doc(data):
    data = [s.strip() for s in data]
    data = [clean_str(s) for s in data]
    return data


def pad_sequences(sentences, padding_word="<PAD/>", pad_len=None):
    if pad_len is not None:
        sequence_length = pad_len
    else:
        sequence_length = max(len(x) for x in sentences)

    padded_sentences = []
    for i in range(len(sentences)):
        sentence = sentences[i]
        num_padding = sequence_length - len(sentence)
        new_sentence = sentence + [padding_word] * num_padding
        padded_sentences.append(new_sentence)
    return padded_sentences


def build_vocab(sentences):
    # Build vocabulary
    word_counts = Counter(itertools.chain(*sentences))
    # Mapping from index to word
    vocabulary_inv = [x[0] for x in word_counts.most_common()]
    # Mapping from word to index
    vocabulary = {x: i for i, x in enumerate(vocabulary_inv)}
    return word_counts, vocabulary, vocabulary_inv


def build_input_data_cnn(sentences, vocabulary):
    x = np.array([[vocabulary[word] for word in sentence] for sentence in sentences])
    return x


def build_input_data_rnn(data, vocabulary, max_doc_len, max_sent_len):
    x = np.zeros((len(data), max_doc_len, max_sent_len), dtype='int32')
    for i, doc in enumerate(data):
        for j, sent in enumerate(doc):
            k = 0
            for word in sent:
                x[i,j,k] = vocabulary[word]
                k += 1
    return x


def extract_keywords(data_path, vocab, class_type, num_keywords, data, perm):
    sup_data = []
    sup_idx = []
    sup_label = []
    file_name = 'doc_id.txt'
    infile = open(join(data_path, file_name), mode='r', encoding='utf-8')
    text = infile.readlines()
    for i, line in enumerate(text):
        line = line.split('\n')[0]
        class_id, doc_ids = line.split(':')
        assert int(class_id) == i
        seed_idx = doc_ids.split(',')
        seed_idx = [int(idx) for idx in seed_idx]
        sup_idx.append(seed_idx)
        for idx in seed_idx:
            sup_data.append(" ".join(data[idx]))
            sup_label.append(i)

    from sklearn.feature_extraction.text import TfidfVectorizer
    import nltk

    tfidf = TfidfVectorizer(norm='l2', sublinear_tf=True, max_df=0.2, stop_words='english')
    sup_x = tfidf.fit_transform(sup_data)
    sup_x = np.asarray(sup_x.todense())

    vocab_dict = tfidf.vocabulary_
    vocab_inv_dict = {v: k for k, v in vocab_dict.items()}

    print("\n### Supervision type: Labeled documents ###")
    print("Extracted keywords for each class: ")
    keywords = []
    cnt = 0
    for i in range(len(sup_idx)):
        class_vec = np.average(sup_x[cnt:cnt+len(sup_idx[i])], axis=0)
        cnt += len(sup_idx[i])
        sort_idx = np.argsort(class_vec)[::-1]
        keyword = []
        if class_type == 'topic':
            j = 0
            k = 0
            while j < num_keywords:
                w = vocab_inv_dict[sort_idx[k]]
                if w in vocab:
                    keyword.append(vocab_inv_dict[sort_idx[k]])
                    j += 1
                k += 1
        elif class_type == 'sentiment':
            j = 0
            k = 0
            while j < num_keywords:
                w = vocab_inv_dict[sort_idx[k]]
                w, t = nltk.pos_tag([w])[0]
                if t.startswith("J") and w in vocab:
                    keyword.append(w)
                    j += 1
                k += 1
        print("Class {}:".format(i))
        print(keyword)
        keywords.append(keyword)

    new_sup_idx = []
    m = {v: k for k, v in enumerate(perm)}
    for seed_idx in sup_idx:
        new_seed_idx = []
        for ele in seed_idx:
            new_seed_idx.append(m[ele])
        new_sup_idx.append(new_seed_idx)
    new_sup_idx = np.asarray(new_sup_idx)

    return keywords, new_sup_idx


def load_keywords(data_path, sup_source):
    if sup_source == 'labels':
        file_name = 'classes.txt'
        print("\n### Supervision type: Label Surface Names ###")
        print("Label Names for each class: ")
    elif sup_source == 'keywords':
        file_name = 'keywords.txt'
        print("\n### Supervision type: Class-related Keywords ###")
        print("Keywords for each class: ")
    infile = open(join(data_path, file_name), mode='r', encoding='utf-8')
    text = infile.readlines()

    keywords = []
    for i, line in enumerate(text):
        line = line.split('\n')[0]
        class_id, contents = line.split(':')
        assert int(class_id) == i
        keyword = contents.split(',')
        print("Supervision content of class {}:".format(i))
        print(keyword)
        keywords.append(keyword)
    return keywords


def load_cnn(dataset_name, sup_source, num_keywords=10, with_evaluation=True, truncate_len=None):
    data_path = './' + dataset_name
    data, y = read_file(data_path, with_evaluation)

    sz = len(data)
    np.random.seed(1234)
    perm = np.random.permutation(sz)

    data = preprocess_doc(data)
    data = [s.split(" ") for s in data]

    tmp_list = [len(doc) for doc in data]
    len_max = max(tmp_list)
    len_avg = np.average(tmp_list)
    len_std = np.std(tmp_list)

    print("\n### Dataset statistics: ###")
    print('Document max length: {} (words)'.format(len_max))
    print('Document average length: {} (words)'.format(len_avg))
    print('Document length std: {} (words)'.format(len_std))

    if truncate_len is None:
        truncate_len = min(int(len_avg + 3*len_std), len_max)
    print("Defined maximum document length: {} (words)".format(truncate_len))
    print('Fraction of truncated documents: {}'.format(sum(tmp > truncate_len for tmp in tmp_list)/len(tmp_list)))

    sequences_padded = pad_sequences(data)
    word_counts, vocabulary, vocabulary_inv = build_vocab(sequences_padded)
    x = build_input_data_cnn(sequences_padded, vocabulary)
    x = x[perm]

    if with_evaluation:
        print("Number of classes: {}".format(len(np.unique(y))))
        print("Number of documents in each class:")
        for i in range(len(np.unique(y))):
            print("Class {}: {}".format(i, len(np.where(y == i)[0])))
        y = y[perm]

    print("Vocabulary Size: {:d}".format(len(vocabulary_inv)))

    if sup_source == 'labels' or sup_source == 'keywords':
        keywords = load_keywords(data_path, sup_source)
        return x, y, word_counts, vocabulary, vocabulary_inv, len_avg, len_std, keywords, perm
    elif sup_source == 'docs':
        if dataset_name == 'nyt':
            class_type = 'topic'
        elif dataset_name == 'News':
            class_type = 'topic'
        elif dataset_name == 'Movies':
            class_type = 'sentiment'
        keywords, sup_idx = extract_keywords(data_path, vocabulary, class_type, num_keywords, data, perm)
        return x, y, word_counts, vocabulary, vocabulary_inv, len_avg, len_std, keywords, sup_idx, perm


def load_rnn(dataset_name, sup_source, num_keywords=10, with_evaluation=True, truncate_len=None):
    data_path = './' + dataset_name
    data, y = read_file(data_path, with_evaluation)

    sz = len(data)
    np.random.seed(1234)
    perm = np.random.permutation(sz)

    data = preprocess_doc(data)
    data_copy = [s.split(" ") for s in data]
    docs_padded = pad_sequences(data_copy)
    word_counts, vocabulary, vocabulary_inv = build_vocab(docs_padded)

    data = [tokenize.sent_tokenize(doc) for doc in data]
    flat_data = [sent for doc in data for sent in doc]

    tmp_list = [len(sent.split(" ")) for sent in flat_data]
    max_sent_len = max(tmp_list)
    avg_sent_len = np.average(tmp_list)
    std_sent_len = np.std(tmp_list)

    print("\n### Dataset statistics: ###")
    print('Sentence max length: {} (words)'.format(max_sent_len))
    print('Sentence average length: {} (words)'.format(avg_sent_len))

    if truncate_len is None:
        truncate_sent_len = min(int(avg_sent_len + 3*std_sent_len), max_sent_len)
    else:
        truncate_sent_len = truncate_len[1]
    print("Defined maximum sentence length: {} (words)".format(truncate_sent_len))
    print('Fraction of truncated sentences: {}'.format(sum(tmp > truncate_sent_len for tmp in tmp_list)/len(tmp_list)))

    tmp_list = [len(doc) for doc in data]
    max_doc_len = max(tmp_list)
    avg_doc_len = np.average(tmp_list)
    std_doc_len = np.std(tmp_list)

    print('Document max length: {} (sentences)'.format(max_doc_len))
    print('Document average length: {} (sentences)'.format(avg_doc_len))

    if truncate_len is None:
        truncate_doc_len = min(int(avg_doc_len + 3*std_doc_len), max_doc_len)
    else:
        truncate_doc_len = truncate_len[0]
    print("Defined maximum document length: {} (sentences)".format(truncate_doc_len))
    print('Fraction of truncated documents: {}'.format(sum(tmp > truncate_doc_len for tmp in tmp_list)/len(tmp_list)))

    len_avg = [avg_doc_len, avg_sent_len]
    len_std = [std_doc_len, std_sent_len]

    data = [[sent.split(" ") for sent in doc] for doc in data]
    x = build_input_data_rnn(data, vocabulary, max_doc_len, max_sent_len)
    x = x[perm]

    if with_evaluation:
        print("Number of classes: {}".format(len(np.unique(y))))
        print("Number of documents in each class:")
        for i in range(len(np.unique(y))):
            print("Class {}: {}".format(i, len(np.where(y == i)[0])))
        y = y[perm]

    print("Vocabulary Size: {:d}".format(len(vocabulary_inv)))

    if sup_source == 'labels' or sup_source == 'keywords':
        keywords = load_keywords(data_path, sup_source)
        return x, y, word_counts, vocabulary, vocabulary_inv, len_avg, len_std, keywords, perm
    elif sup_source == 'docs':
        if dataset_name == 'nyt':
            class_type = 'topic'
        elif dataset_name == 'News':
            class_type = 'topic'
        elif dataset_name == 'Movies':
            class_type = 'sentiment'
        keywords, sup_idx = extract_keywords(data_path, vocabulary, class_type, num_keywords, data_copy, perm)
        return x, y, word_counts, vocabulary, vocabulary_inv, len_avg, len_std, keywords, sup_idx, perm


def load_dataset(dataset_name, sup_source, model='cnn', with_evaluation=True, truncate_len=None):
    if model == 'cnn':
        return load_cnn(dataset_name, sup_source, with_evaluation=with_evaluation, truncate_len=truncate_len)
    elif model == 'rnn':
        return load_rnn(dataset_name, sup_source, with_evaluation=with_evaluation, truncate_len=truncate_len)


In [12]:
import numpy as np
import os
np.random.seed(1234)
from sklearn.cluster import KMeans
from spherecluster import SphericalKMeans
from spherecluster import *
def seed_expansion(word_sup_array, prob_sup_array, sz, write_path, vocabulary_inv, embedding_mat):
    expanded_seed = []
    vocab_sz = len(vocabulary_inv)
    for j, word_class in enumerate(word_sup_array):
        prob_sup_class = prob_sup_array[j]
        expanded_class = []
        seed_vec = np.zeros(vocab_sz)
        if len(word_class) < sz:
            for i, word in enumerate(word_class):
                seed_vec[word] = prob_sup_class[i]
            expanded = np.dot(embedding_mat.transpose(), seed_vec)
            expanded = np.dot(embedding_mat, expanded)
            word_expanded = sorted(range(len(expanded)), key=lambda k: expanded[k], reverse=True)
            for i in range(sz):
                expanded_class.append(word_expanded[i])
            expanded_seed.append(np.array(expanded_class))
        else:
            expanded_seed.append(word_class)
        if write_path is not None:
            if not os.path.exists(write_path):
                os.makedirs(write_path)
            f = open(write_path + 'class' + str(j) + '_' + str(sz) + '.txt', 'w')
            for i, word in enumerate(expanded_class):
                f.write(vocabulary_inv[word] + ' ')
            f.close()
    return expanded_seed


def label_expansion(class_labels, write_path, vocabulary_inv, embedding_mat):
    print("Retrieving top-t nearest words...")
    n_classes = len(class_labels)
    prob_sup_array = []
    current_szes = []
    all_class_labels = []
    for class_label in class_labels:
        current_sz = len(class_label)
        current_szes.append(current_sz)
        prob_sup_array.append([1/current_sz] * current_sz)
        all_class_labels += list(class_label)
    current_sz = np.min(current_szes)
    while len(all_class_labels) == len(set(all_class_labels)):
        current_sz += 1
        expanded_array = seed_expansion(class_labels, prob_sup_array, current_sz, None, vocabulary_inv, embedding_mat)
        all_class_labels = [w for w_class in expanded_array for w in w_class]

    expanded_array = seed_expansion(class_labels, prob_sup_array, current_sz-1, None, vocabulary_inv, embedding_mat)
    print("Final expansion size t = {}".format(len(expanded_array[0])))

    centers = []
    kappas = []
    print("Top-t nearest words for each class:")
    for i in range(n_classes):
        expanded_class = expanded_array[i]
        vocab_expanded = [vocabulary_inv[w] for w in expanded_class]
        print("Class {}:".format(i))
        print(vocab_expanded)
        expanded_mat = embedding_mat[np.asarray(expanded_class)]
        vmf_soft = VonMisesFisherMixture(n_clusters=1, n_jobs=15)
        vmf_soft.fit(expanded_mat)
        center = vmf_soft.cluster_centers_[0]
        kappa = vmf_soft.concentrations_[0]
        centers.append(center)
        kappas.append(kappa)

    for j, expanded_class in enumerate(expanded_array):
        if write_path is not None:
            if not os.path.exists(write_path):
                os.makedirs(write_path)
            f = open(write_path + 'class' + str(j) + '.txt', 'w')
            for i, word in enumerate(expanded_class):
                f.write(vocabulary_inv[word] + ' ')
            f.close()
    print("Finished vMF distribution fitting.")
    return expanded_array, centers, kappas


def pseudodocs(word_sup_array, total_num, background_array, sequence_length, len_avg,
                len_std, num_doc, interp_weight, vocabulary_inv, embedding_mat, model, save_dir=None):

    for i in range(len(embedding_mat)):
        embedding_mat[i] = embedding_mat[i] / np.linalg.norm(embedding_mat[i])

    _, centers, kappas = \
    label_expansion(word_sup_array, save_dir, vocabulary_inv, embedding_mat)
    print(word_sup_array)
    print("Pseudo documents generation...")
    background_vec = interp_weight * background_array
    if model == 'cnn':
        docs = np.zeros((num_doc*len(word_sup_array), sequence_length), dtype='int32')
        label = np.zeros((num_doc*len(word_sup_array), len(word_sup_array)))
        for i in range(len(word_sup_array)):
            docs_len = len_avg*np.ones(num_doc)
            center = centers[i]
            kappa = kappas[i]
            discourses = sample_vMF(center, kappa, num_doc)
            for j in range(num_doc):
                discourse = discourses[j]
                prob_vec = np.dot(embedding_mat, discourse)
                prob_vec = np.exp(prob_vec)
                sorted_idx = np.argsort(prob_vec)[::-1]
                delete_idx = sorted_idx[total_num:]
                prob_vec[delete_idx] = 0
                prob_vec /= np.sum(prob_vec)
                prob_vec *= 1 - interp_weight
                prob_vec += background_vec
                doc_len = int(docs_len[j])
                docs[i*num_doc+j][:doc_len] = np.random.choice(len(prob_vec), size=doc_len, p=prob_vec)
                label[i*num_doc+j] = interp_weight/len(word_sup_array)*np.ones(len(word_sup_array))
                label[i*num_doc+j][i] += 1 - interp_weight
    elif model == 'rnn':
        docs = np.zeros((num_doc*len(word_sup_array), sequence_length[0], sequence_length[1]), dtype='int32')
        label = np.zeros((num_doc*len(word_sup_array), len(word_sup_array)))
        doc_len = int(len_avg[0])
        sent_len = int(len_avg[1])
        for period_idx in vocabulary_inv:
            if vocabulary_inv[period_idx] == '.':
                break
        for i in range(len(word_sup_array)):
            center = centers[i]
            kappa = kappas[i]
            discourses = sample_vMF(center, kappa, num_doc)
            for j in range(num_doc):
                discourse = discourses[j]
                prob_vec = np.dot(embedding_mat, discourse)
                prob_vec = np.exp(prob_vec)
                sorted_idx = np.argsort(prob_vec)[::-1]
                delete_idx = sorted_idx[total_num:]
                prob_vec[delete_idx] = 0
                prob_vec /= np.sum(prob_vec)
                prob_vec *= 1 - interp_weight
                prob_vec += background_vec
                for k in range(doc_len):
                    docs[i*num_doc+j][k][:sent_len] = np.random.choice(len(prob_vec), size=sent_len, p=prob_vec)
                    docs[i*num_doc+j][k][sent_len] = period_idx
                label[i*num_doc+j] = interp_weight/len(word_sup_array)*np.ones(len(word_sup_array))
                label[i*num_doc+j][i] += 1 - interp_weight

    print("Finished Pseudo documents generation.")
    print("docs",docs)
    print("label",label)
    return docs, label


def augment(x, sup_idx, total_len):
    print("Labeled documents augmentation...")
    docs = x[sup_idx.flatten()]
    curr_len = len(docs)
    copy_times = int(total_len/curr_len) - 1
    y = np.zeros(len(sup_idx.flatten()), dtype='int32')
    label_nums = [len(seed_idx) for seed_idx in sup_idx]
    cnt = 0
    for i in range(len(sup_idx)):
        y[cnt:cnt+label_nums[i]] = i
        cnt += label_nums[i]

    new_docs = docs
    new_y = y
    for i in range(copy_times):
        new_docs = np.concatenate((new_docs, docs), axis=0)
        new_y = np.concatenate((new_y, y), axis=0)

    pretrain_labels = np.zeros((len(new_y),len(np.unique(y))))
    for i in range(len(new_y)):
        pretrain_labels[i][new_y[i]] = 1.0

    print("Finished labeled documents augmentation.")
    return new_docs, pretrain_labels


In [13]:
import numpy as np
np.random.seed(1234)
import os
from time import time
import csv
import keras.backend as K
# K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=30, inter_op_parallelism_threads=30)))
from keras.layers import Layer
from keras.layers import Dense, Input, Convolution1D, Embedding, GlobalMaxPooling1D, GRU, TimeDistributed
from keras.layers import Concatenate
from keras.models import Model
from keras import initializers, regularizers, constraints
from keras.initializers import VarianceScaling, RandomUniform
from sklearn.metrics import f1_score


def f1(y_true, y_pred):
    y_true = y_true.astype(np.int64)
    assert y_pred.size == y_true.size
    f1_macro = f1_score(y_true, y_pred, average='macro')
    f1_micro = f1_score(y_true, y_pred, average='micro')
    return f1_macro, f1_micro


def ConvolutionLayer(input_shape, n_classes, filter_sizes=[2, 3, 4, 5], num_filters=20, word_trainable=False, vocab_sz=None,
                     embedding_matrix=None, word_embedding_dim=100, hidden_dim=20, act='relu', init='ones'):
    x = Input(shape=(input_shape,), name='input')
    z = Embedding(vocab_sz, word_embedding_dim, input_length=(input_shape,), name="embedding",
                    weights=[embedding_matrix], trainable=word_trainable)(x)

    conv_blocks = []
    for sz in filter_sizes:
        conv = Convolution1D(filters=num_filters,
                             kernel_size=sz,
                             padding="valid",
                             activation=act,
                             strides=1,
                             kernel_initializer=init)(z)
        conv = GlobalMaxPooling1D()(conv)
        conv_blocks.append(conv)
    z = Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]
    z = Dense(hidden_dim, activation="relu")(z)
    y = Dense(n_classes, activation="softmax")(z)
    return Model(inputs=x, outputs=y, name='classifier')


def dot_product(x, kernel):
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)


class AttentionWithContext(Layer):
    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 init='glorot_uniform', bias=True, **kwargs):

        self.supports_masking = True
        self.init = init

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight(shape=(input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]


def HierAttLayer(input_shape, n_classes, word_trainable=False, vocab_sz=None,
                embedding_matrix=None, word_embedding_dim=100, gru_dim=100, fc_dim=100):
    sentence_input = Input(shape=(input_shape[2],), dtype='int32')
    embedded_sequences = Embedding(vocab_sz,
                                    word_embedding_dim,
                                    input_length=input_shape[2],
                                    weights=[embedding_matrix],
                                    trainable=word_trainable)(sentence_input)
    l_lstm = GRU(gru_dim, return_sequences=True)(embedded_sequences)
    l_dense = TimeDistributed(Dense(fc_dim))(l_lstm)
    l_att = AttentionWithContext()(l_dense)
    sentEncoder = Model(sentence_input, l_att)

    x = Input(shape=(input_shape[1], input_shape[2]), dtype='int32')
    review_encoder = TimeDistributed(sentEncoder)(x)
    l_lstm_sent = GRU(gru_dim, return_sequences=True)(review_encoder)
    l_dense_sent = TimeDistributed(Dense(fc_dim))(l_lstm_sent)
    l_att_sent = AttentionWithContext()(l_dense_sent)
    y = Dense(n_classes, activation='softmax')(l_att_sent)

    return Model(inputs=x, outputs=y, name='classifier')


class WSTC(object):
    def __init__(self,
                 input_shape,
                 n_classes=None,
                 init=RandomUniform(minval=-0.01, maxval=0.01),
                 y=None,
                 model='cnn',
                 vocab_sz=None,
                 word_embedding_dim=100,
                 embedding_matrix=None
                 ):

        super(WSTC, self).__init__()

        self.input_shape = input_shape
        self.y = y
        self.n_classes = n_classes
        if model == 'cnn':
            self.classifier = ConvolutionLayer(self.input_shape[1], n_classes=n_classes,
                                                vocab_sz=vocab_sz, embedding_matrix=embedding_matrix,
                                                word_embedding_dim=word_embedding_dim, init=init)
        elif model == 'rnn':
            self.classifier = HierAttLayer(self.input_shape, n_classes=n_classes,
                                             vocab_sz=vocab_sz, embedding_matrix=embedding_matrix,
                                             word_embedding_dim=word_embedding_dim)

        self.model = self.classifier
        self.sup_list = {}

    def pretrain(self, x, pretrain_labels, sup_idx=None, optimizer='adam',
                 loss='kld', epochs=200, batch_size=256, save_dir=None):

        self.classifier.compile(optimizer=optimizer, loss=loss)
        print("\nNeural model summary: ")
        self.model.summary()

        if sup_idx is not None:
            for i, seed_idx in enumerate(sup_idx):
                for idx in seed_idx:
                    self.sup_list[idx] = i

        # begin pretraining
        t0 = time()
        print('\nPretraining...')
        self.classifier.fit(x, pretrain_labels, batch_size=batch_size, epochs=epochs)
        print('Pretraining time: {:.2f}s'.format(time() - t0))
        if save_dir is not None:
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
            self.classifier.save_weights(save_dir + '/pretrained.h5')
            print('Pretrained model saved to {}/pretrained.h5'.format(save_dir))
        self.pretrained = True

    def load_weights(self, weights):
        self.model.load_weights(weights)

    def predict(self, x):
        q = self.model.predict(x, verbose=0)
        return q.argmax(1)

    def target_distribution(self, q, power=2):
        weight = q**power / q.sum(axis=0)
        p = (weight.T / weight.sum(axis=1)).T
        for i in self.sup_list:
            p[i] = 0
            p[i][self.sup_list[i]] = 1
        return p

    def compile(self, optimizer='sgd', loss='kld'):
        self.model.compile(optimizer=optimizer, loss=loss)

    def fit(self, x, y=None, maxiter=5e4, batch_size=256, tol=0.1, power=2,
            update_interval=140, save_dir=None, save_suffix=''):

        print('Update interval: {}'.format(update_interval))

        pred = self.classifier.predict(x)
        y_pred = np.argmax(pred, axis=1)
        y_pred_last = np.copy(y_pred)

        # logging file
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        logfile = open(save_dir + '/self_training_log_{}.csv'.format(save_suffix), 'w')
        logwriter = csv.DictWriter(logfile, fieldnames=['iter', 'f1_macro', 'f1_micro'])
        logwriter.writeheader()

        index = 0
        index_array = np.arange(x.shape[0])
        for ite in range(int(maxiter)):
            if ite % update_interval == 0:
                q = self.model.predict(x, verbose=0)

                y_pred = q.argmax(axis=1)
                p = self.target_distribution(q, power)
                print('\nIter {}: '.format(ite), end='')
                if y is not None:
                    f1_macro, f1_micro = np.round(f1(y, y_pred), 5)
                    logdict = dict(iter=ite, f1_macro=f1_macro, f1_micro=f1_micro)
                    logwriter.writerow(logdict)
                    print('f1_macro = {}, f1_micro = {}'.format(f1_macro, f1_micro))

                # check stop criterion
                delta_label = np.sum(y_pred != y_pred_last).astype(np.float) / y_pred.shape[0]
                y_pred_last = np.copy(y_pred)
                print('Fraction of documents with label changes: {} %'.format(np.round(delta_label*100, 3)))
                if ite > 0 and delta_label < tol/100:
                    print('\nFraction: {} % < tol: {} %'.format(np.round(delta_label*100, 3), tol))
                    print('Reached tolerance threshold. Stopping training.')
                    logfile.close()
                    break

            # train on batch
            idx = index_array[index * batch_size: min((index+1) * batch_size, x.shape[0])]
            self.model.train_on_batch(x=x[idx], y=p[idx])
            index = index + 1 if (index + 1) * batch_size <= x.shape[0] else 0

            ite += 1

        logfile.close()

        if save_dir is not None:
            self.model.save_weights(save_dir + '/final.h5')
            print("Final model saved to: {}/final.h5".format(save_dir))
        return self.predict(x)


In [ ]:
! chmod +x test.sh


Movies_Training_Step3

In [8]:
%cd /content/drive/MyDrive/WeSTClass/

!pip install transformers


/content/drive/MyDrive/WeSTClass


ModuleNotFoundError: ignored

In [ ]:
#alpha:  Background Word Distribution Weight, it plays a role in generating psudeo documents considering the
# amount of topic words and back-ground words used. Having a low alpha results in focusing only on topic-words neglecting the background content and can also overfit the model
#ideal range can be vary around 0.85.

In [ ]:
!./test.sh

2023-10-30 03:11:59.317053: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-30 03:11:59.317151: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-30 03:11:59.317208: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-30 03:11:59.333109: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-30 03:12:00.744413: W tensorflow/compiler/

In [ ]:
%cd /content/drive/MyDrive/WeSTClass/Movies
import pandas as pd
import numpy as np
y_pred = []
with open("out.txt","r") as pred:
  line = pred.readlines()
  for i in line:
    y_pred.append(int(i))
data = np.array(pd.read_csv("Movies_Val.csv", header = None))
y= np.array(data[:,0], dtype = np.int8)
from sklearn.metrics import f1_score

f1 = f1_score(y, y_pred[:100])
f1

/content/drive/MyDrive/WeSTClass/Movies


0.7610619469026548

In [ ]:
input_path = '/content/drive/MyDrive/WeSTClass/Movies/out.txt'
output_path = '/content/drive/MyDrive/WeSTClass/Movies/test_pred_movie.txt'

# Read the documents from the input file
with open(input_path, 'r', encoding='utf-8') as file:
    documents = file.readlines()

# Extract the last 1000 documents
last_1000_docs = documents[-25000:]

# Save the last 1000 documents to the output file
with open(output_path, 'w', encoding='utf-8') as file:
    file.writelines(last_1000_docs)

In [ ]:
from gensim.models import KeyedVectors
import numpy as np
embedding_file = "/content/drive/MyDrive/CatE/datasets/News/emb_category_w.txt"
word_vectors = KeyedVectors(vector_size=100)
with open(embedding_file, 'r', encoding='utf-8') as f:
  for line in f:
    parts = line.split()
    word = parts[0]
    vector = np.array([float(x) for x in parts[1:]], dtype=np.float32)
    word_vectors.add_vector(word, vector)
word_vectors.save_word2vec_format("custom_word2vec_format.bin", binary=True)

/usr/local/lib/python3.10/dist-packages/gensim/models/keyedvectors.py:551: UserWarning: Adding single vectors to a KeyedVectors which grows by one each time can be costly. Consider adding in batches or preallocating to the required size.
  warnings.warn(


In [ ]:
%cd /content/drive/MyDrive/WeSTClass/
!./test.sh

/content/drive/MyDrive/WeSTClass
2023-10-28 07:17:47.416824: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-28 07:17:47.416918: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-28 07:17:47.416977: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-28 07:17:47.431036: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-28 07:17:

In [ ]:
%cd /content/drive/MyDrive/WeSTClass/Movies
import pandas as pd
import numpy as np
y_pred = []
with open("out.txt","r") as pred:
  line = pred.readlines()
  for i in line:
    y_pred.append(int(i))
data = np.array(pd.read_csv("Movies_Val.csv", header = None))
y= np.array(data[:,0], dtype = np.int8)
from sklearn.metrics import f1_score

f1 = f1_score(y, y_pred[:100])
f1

/content/drive/MyDrive/WeSTClass/Movies


0.787878787878788

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.1 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model = "facebook/bart-large-mnli")
labels = ["politics","sports","business","technology"]

In [ ]:
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/WeSTClass/News/combined_file.csv", header = None,error_bad_lines=False)
data.columns = ["actual_labels","document"]

<ipython-input-4-2edb653bfbb9>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv("/content/drive/MyDrive/WeSTClass/News/combined_file.csv", header = None,error_bad_lines=False)


In [ ]:
from tqdm import tqdm
for i in tqdm(range(4000,6000)):
  sequence= data.iloc[i,1]
  data.loc[i,"pseudo_label_bart-l-m"]=classifier(sequence,labels)["labels"][0]
data.to_csv("/content/drive/MyDrive/WeSTClass/News/combined_file_latest5.csv")

100%|██████████| 2000/2000 [2:27:52<00:00,  4.44s/it]
